## Sumário

1. [Introdução](#1.-Introdução)
2. [EDA e Feature Engineering](#2.-EDA-e-Feature-Engineering)
3. [Clustering](#3.-Clustering)

   [Referências](#Referências)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore'
)

## 1. Introdução

Esse notebook tem como objetivo fornecer uma solução para o case proposto. Dessa forma, busca-se executar as atividades abaixo:

1. Classificar e caracterizar os municípios com base nos dados demográficos;
2. Identificar e justificar quais os municípios mais adequados para a entrada da empresa no mercado brasileiro;
3. Elaborar um modelo de classificação para identificar a qual grupo o município pertence.

São feitas algumas considerações sobre qual seria o ambiente ideal para produzir a população ideal para consumir bebidas industrializadas.

1. Conforme [1] e [2], o consumo de bebidas industrializadas é mais prevalente dentre a população mais jovem. Dessa forma, busca-se cidades que possuam alta taxa de crescimento populacional e um bom percentual de pessoas até 25 anos.

2. Conforme [3], o consumo de bebidas industrializadas é mais prevalente dentre a população de maior renda. Dessa forma busca-se cidades que possuam renda per capita alta e baixos indices de pobreza.

3. Entende-se que grandes cidades são adequadas por possuirem um potencial de clientes maior, além de possuir mais mão de obra para contratação, visto que a empresa ainda não está implantada. Dessa forma, busca-se cidades com alta taxa de crescimento populacional e grande quantidade de população.

4. Entende-se que populações mais urbanas apresentam maior facilidade para aceitar novos produtos. Dessa forma, busca-se cidades que tenham alto percentual de população urbana e que estejam próximas das capitais.

5. Por último, vale olhar para cases recentes de outros players, tal como o lançamento do Clash'd pela Heineken [4]. Os estados escolhidos pelo player concorrente devem ser levadas em consideração visto que provavelmente houve um extenso estudo antes da escolha.

Destaca-se que os dados das pesquisas não são recentes, no entanto os dados demográficos disponíveis também datam de 1991 e 2000, portanto há essa ressalva quanto a validade da análise desenvolvida.

## 2. EDA e Feature Engineering

Seleciona-se apenas as colunas de interesse conforme as considerações feitas na introdução. As colunas são renomeadas para facilitar o uso.

As features que serão utilizadas são:

1. Distância da capital
2. Taxa de fecundidade total em 2000
3. Renda per Capita
4. Intensidade de pobreza
5. % de População até 25 anos
6. População de até 25 anos em 2000
7. Taxa de crescimento da população de até 25 anos
8. % de População urbana
9. Estados em que o concorrente optou por lançar seu produto

Das quais as features 1 a 4 já estão prontas no dataset original e as features 5 a 9 serão obtidas.

In [2]:
cities_raw = pd.read_excel('data/raw_data.xls')

cities = cities_raw[[
    'Código',
    'Município',
    'UF',
    'Distância à capital (km)',
    'Taxa de fecundidade total, 2000',
    'Renda per Capita, 2000',
    'Intensidade da pobreza, 2000',
    'População de 25 anos ou mais de idade, 2000',
    'População de 25 anos ou mais de idade, 1991',
    'População total, 2000',
    'População total, 1991',
    'População urbana, 2000'
]]

cities.rename(columns={
    'Código': 'id',
    'Município': 'municipio',
    'UF': 'uf',
    'Distância à capital (km)': 'dist_cap_km',
    'Taxa de fecundidade total, 2000': 'taxa_fec',
    'Renda per Capita, 2000': 'renda_cap',
    'Intensidade da pobreza, 2000': 'pobreza',
    'População de 25 anos ou mais de idade, 2000': 'pop_25_mais_2000',
    'População de 25 anos ou mais de idade, 1991': 'pop_25_mais_1991',
    'População total, 2000': 'pop_total_2000',
    'População total, 1991': 'pop_total_1991',
    'População urbana, 2000': 'pop_urb_2000'
}, inplace=True)

cities.set_index(['id'], inplace=True)

Nota-se que não existem colunas com dados faltantes. Pelos boxplots percebe-se que existem outliers, mas isso é natual do dataset, existe a tendência de concentração de pessoas em grandes cidades, por exemplo. Dessa forma, não será necessário esforço para o tratamento dos dados.

In [3]:
cities.describe()

,dist_cap_km,taxa_fec,renda_cap,pobreza,pop_25_mais_2000,pop_25_mais_1991,pop_total_2000,pop_total_1991,pop_urb_2000
count,5507.000000,5507.000000,5507.000000,5507.000000,5.507000e+03,5.507000e+03,5.507000e+03,5.507000e+03,5.507000e+03
mean,253.212620,2.864845,170.814160,47.086060,1.551693e+04,1.221264e+04,3.083333e+04,2.666167e+04,2.505066e+04
std,163.210532,0.744454,96.425347,10.732511,1.039028e+05,8.923783e+04,1.867506e+05,1.690004e+05,1.800123e+05
min,0.000000,1.560000,28.380000,15.760000,4.220000e+02,2.390000e+02,7.950000e+02,6.180000e+02,0.000000e+00
25%,121.858906,2.320000,86.495000,38.190000,2.502000e+03,2.119500e+03,5.152500e+03,4.945000e+03,2.402500e+03
50%,228.262939,2.670000,159.100000,46.230000,4.930000e+03,4.084000e+03,1.041800e+04,9.748000e+03,5.314000e+03
75%,358.072044,3.230000,232.695000,55.410000,1.004000e+04,8.104000e+03,2.135650e+04,1.897450e+04,1.324050e+04
max,1474.314590,7.790000,954.650000,83.030000,5.825893e+06,5.106165e+06,1.043425e+07,9.649519e+06,9.813187e+06


In [4]:
# Comentado para facilitar a leitura do documento. Pode ser descomentado durante análises detalhadas.

# for column in cities.select_dtypes(exclude=['object']):
#     plt.figure()
#     cities.boxplot([column])

In [5]:
cities['pop_ate_25_2000'] = cities['pop_total_2000'] - cities['pop_25_mais_2000']
cities['pop_ate_25_1991'] = cities['pop_total_1991'] - cities['pop_25_mais_1991']

cities['perc_ate_25_2000'] = cities['pop_ate_25_2000'] / cities['pop_total_2000']
cities['var_pop_ate_25'] = cities['pop_ate_25_2000'] / cities['pop_ate_25_1991']

cities['perc_pop_urb'] = cities['pop_urb_2000'] / cities['pop_total_2000']

cities['player'] = np.where(cities['uf'].isin(['SP', 'RJ', 'MG', 'PR']), 1, 0)

cities = cities[[
    'dist_cap_km',
    'taxa_fec',
    'renda_cap',
    'pobreza',
    'pop_ate_25_2000',
    'perc_ate_25_2000',
    'var_pop_ate_25',
    'perc_pop_urb',
    'player'
]]

cities.to_csv('data/cities.csv', index=True)

## 3. Clustering

## Referências

[1. Pesquisa AFEBRAS - Idade](https://afrebras.org.br/noticias/pesquisa-revela-quem-consome-mais-refrigerante-no-pais/)

[2. Artigo UFPA - Idade](https://coesa.ufpa.br/arquivos/2018/expandido/pesquisa/epidemiologia/PES207.pdf)

[3. Pesquisa IBGE - Renda](https://www.uol.com.br/vivabem/noticias/redacao/2011/07/28/brasileiros-consomem-mais-de-15-milhoes-de-litros-de-refrigerante-por-dia-aponta-ibge.htm)

[4. Lançamento concorrente](https://www.heinekenbrasil.com.br/noticias/clash-d-grupo-heineken-inova-ao-lancar-primeiro-refrigerante-brewed-do-mercado-brasileiro/#)